### Load Libraries

In [1]:
import os
import sys

import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests

/tmp/ipykernel_30032/3715504500.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Enter Spotify credentials

In [91]:
spotify_key = os.getenv('SPOTIFY_KEY')
spotify_secret = os.getenv('SPOTIFY_SECRET')

client_credentials_manager = SpotifyClientCredentials(client_id=spotify_key, 
                                                    client_secret=spotify_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [7]:
top_50_us_playlist_uri = '37i9dQZEVXbLRQDuF5jeBp'

In [8]:
top_50_global_playlist_uri= '37i9dQZEVXbMDoHDwVN2tF'

### Pull songs from playlists

In [9]:
for x in sp.playlist_tracks(top_50_us_playlist_uri)["items"]:
    print(x)

{'added_at': '2024-02-13T23:36:52Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/'}, 'href': 'https://api.spotify.com/v1/users/', 'id': '', 'type': 'user', 'uri': 'spotify:user:'}, 'is_local': False, 'primary_color': None, 'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5K4W6rqBFWDnAN6FQUkS6x'}, 'href': 'https://api.spotify.com/v1/artists/5K4W6rqBFWDnAN6FQUkS6x', 'id': '5K4W6rqBFWDnAN6FQUkS6x', 'name': 'Kanye West', 'type': 'artist', 'uri': 'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/7c0XG5cIJTrrAgEC3ULPiq'}, 'href': 'https://api.spotify.com/v1/artists/7c0XG5cIJTrrAgEC3ULPiq', 'id': '7c0XG5cIJTrrAgEC3ULPiq', 'name': 'Ty Dolla $ign', 'type': 'artist', 'uri': 'spotify:artist:7c0XG5cIJTrrAgEC3ULPiq'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', '

In [118]:
playlist_top_50_us_dict = sp.playlist_tracks(top_50_us_playlist_uri)["items"]

In [119]:
tracks = []

def get_top_50_playlist_tracks(playist: dict) -> pd.DataFrame:

    for track in playlist_top_50_us_dict:

        # Track ID
        track_id = track['track']['id']
        # Track URI
        track_uri = track['track']['uri']
        # Track Name
        track_name = track['track']['name']
        # Artist Info
        artists = track['track']['artists']
        track_artists = []
        tract_artist_id = []
        for artist in artists:
            artist_name = artist['name']
            artist_id = artist['id']
            track_artists.append([artist_name, artist_id])
        # Track Popularity
        track_popularity = track['track']['popularity']
        # Is Track Explicit
        track_explicit = track['track']['explicit']
        # Track Duration (ms)
        track_duration_ms  = track['track']['duration_ms']

        # Append results to list
        tracks.append([track_id, track_uri, track_name, track_artists, track_popularity, track_explicit, track_duration_ms])

        # Tracks DataFrame
        tracks_df_cols = ['track_id', 'track_uri', 'track_name', 'track_artists', 'track_popularity', 'track_explicit', 'track_duration_ms']
        tracks_df = pd.DataFrame(tracks, columns=tracks_df_cols)

    return tracks_df

In [120]:
tracks_df = get_top_50_playlist_tracks(playlist_top_50_us_dict)

In [121]:
tracks_df.head()

,track_id,track_uri,track_name,track_artists,track_popularity,track_explicit,track_duration_ms
0,4m4BHvDfzIgj5LbQLuFiIp,spotify:track:4m4BHvDfzIgj5LbQLuFiIp,CARNIVAL,"[[Kanye West, 5K4W6rqBFWDnAN6FQUkS6x], [Ty Dol...",82,True,264324
1,0Z7nGFVCLfixWctgePsRk9,spotify:track:0Z7nGFVCLfixWctgePsRk9,TEXAS HOLD 'EM,"[[Beyoncé, 6vWDO969PvNqNYHIOW5v0m]]",0,True,235636
2,5rlXuCLnMdfo85079qXHoz,spotify:track:5rlXuCLnMdfo85079qXHoz,FUK SUMN,"[[Kanye West, 5K4W6rqBFWDnAN6FQUkS6x], [Ty Dol...",80,True,209577
3,6tNQ70jh4OwmPGpYy6R2o9,spotify:track:6tNQ70jh4OwmPGpYy6R2o9,Beautiful Things,"[[Benson Boone, 22wbnEMDvgVIAGdFeek6ET]]",96,False,180304
4,0mflMxspEfB0VbI1kyLiAv,spotify:track:0mflMxspEfB0VbI1kyLiAv,Stick Season,"[[Noah Kahan, 2RQXRUsr4IW1f3mKyKsy4B]]",98,False,182346


In [122]:
# Saving track ID to list to pull audio data
all_track_uris = tracks_df['track_uri'].to_list()

In [139]:
# Audio Features list
audio_features = []

for track_uri in all_track_uris[0:4]:
    feat = sp.audio_features(track_uri)[0]
    track_acousticness = feat['acousticness']
    track_danceability = feat['danceability']
    track_energy = feat['energy']
    track_speechiness = feat['speechiness']
    track_instrumentalness = feat['instrumentalness']
    track_loudness = feat['loudness']
    track_tempo = feat['tempo']
    track_liveness = feat['liveness']
    track_valence = feat['valence']

    analysis = sp.audio_analysis(track_uri)['track']
    track_time_signature = analysis['time_signature']
    track_key = analysis['key']

    # Append results to list
    audio_features.append([track_uri, track_acousticness, track_danceability, track_energy, track_speechiness, track_instrumentalness, track_loudness, track_tempo, track_liveness, track_valence, track_time_signature, track_key])

    # Features DataFrame
    features_df_cols = ['track_uri', 'track_acousticness', 'track_danceability', 'track_energy', 'track_speechiness', 'track_instrumentalness', 'track_loudness', 'track_tempo', 'track_liveness', 'track_valence', 'track_time_signature', 'track_key']
    features_df = pd.DataFrame(audio_features, columns=features_df_cols)

In [140]:
features_df.head()

,track_uri,track_acousticness,track_danceability,track_energy,track_speechiness,track_instrumentalness,track_loudness,track_tempo,track_liveness,track_valence,track_time_signature,track_key
0,spotify:track:4m4BHvDfzIgj5LbQLuFiIp,0.192,0.596,0.810,0.1520,0,-5.745,148.241,0.338,0.290,4,1
1,spotify:track:0Z7nGFVCLfixWctgePsRk9,0.588,0.725,0.709,0.0720,0,-6.514,110.024,0.135,0.353,4,2
2,spotify:track:5rlXuCLnMdfo85079qXHoz,0.132,0.775,0.817,0.1470,0,-2.825,142.092,0.053,0.463,4,4
3,spotify:track:6tNQ70jh4OwmPGpYy6R2o9,0.151,0.472,0.471,0.0603,0,-5.692,105.029,0.140,0.219,3,10
